# **2022 ML FALL HW1: PM2.5 Prediction (Regression)**

Author: MLTAs

Methods:
* Training with all data
* Training config: mini-batch=512, optimizer=Adam, learning rate=0.1 (TODO: Change the config!)



# **Import Some Packages**

In [138]:
import numpy as np
import csv
import math
import pandas as pd

In [139]:
import os
os.getcwd()

'd:\\NTU\\First Year\\Machine Learning\\HW\\HW1\\Programming'

# **Fix random seed**


This is for the reproduction of your result. **DO NOT modify this secton!** 


In [140]:
import random

seed = 9487
random.seed(seed)

# **Download training data**


In [141]:
# !gdown --id "1Hfzrcm69QwdFvdeF0uASoQlcVxKw_hHy" --output "train.csv"s
# !gdown --id '1FXJztppYG9Q4b_4NHvcmPsc4o5obceWC' --output "test.csv"

# Incase the links above die, you can use the following instead.
#!gdown --id '11abE854Eyv4BA7qt5k8r_80sJ3KuOQUN' --output "train.csv"
# !gdown --id '1uod-Z4ztluXnuHtgUbm39nMudUKqXHMl' --output "test.csv"

# If the data is still missing, you can manually download it from kaggle, and upload the files under /content

# **Define function**

In [142]:
def valid(x, y):
  # TODO: Try to filter out extreme values.
  # ex: If PM2.5 > 100, then we don't use the data to train (return False), otherwise return True,

  # Total unvalid datas is 24
  global count
  if x[1][0] > 1.2 or x[2][0] > 40 or x[3][0] > 50 or x[4][0] > 80 or x[6][0] > 200 or x[14][0] > 70:
    count += 1
    return False
  return True


# Create your dataset
def parse2train(data, feats):

  x = []
  y = []

  # Use data #0~#7 to predict #8 => Total data length should be decresased by 8.
  total_length = data.shape[1] - 8  # data.shape[1] == 5774
  
  for i in range(total_length):
    x_tmp = data[feats, i:i+8] # Use data #0~#7 to predict #8, data #1~#8 to predict #9, etc. Feats is row of data and i:i+8 is column of data
    y_tmp = data[-1, i+8] # last column of (i+8)th row: PM2.5

    # Filter out extreme values to train.
    if valid(x_tmp, y_tmp):
      x.append(x_tmp.reshape(-1,))
      y.append(y_tmp)
  

  # Total valid datas is 5742
  # x.shape: (n, 15, 8)
  # y.shape: (n, 1) 
  x = np.array(x)
  y = np.array(y)

  return x,y


# **Adam**
* This is our gradient descent algorithm. Adam was implemented.
* You can implement another algorithm such as SGD, which may (or may not) boost the performance.
* However, **modules like sklearn and pytorch are not allowed**.
* Ref: https://arxiv.org/pdf/1412.6980.pdf
![](https://i.imgur.com/jRaebdf.png)



In [143]:
# TODO: Implement 2-nd polynomial regression version for the report.
def minibatch(x, y, config):
  
    # Randomize the data in minibatch
    index = np.arange(x.shape[0])
    np.random.shuffle(index)
    x = x[index]
    y = y[index]
    
    # Initialization
    batch_size = config.batch_size
    lr = config.lr
    lam = config.lam
    epoch = config.epoch

    beta_1 = np.full(x[0].shape, 0.9).reshape(-1, 1)
    beta_2 = np.full(x[0].shape, 0.99).reshape(-1, 1)
    # Linear regression: only contains two parameters (w, b).
    w = np.full(x[0].shape, 0.1).reshape(-1, 1)
    bias = 0.1
    m_t = np.full(x[0].shape, 0).reshape(-1, 1)
    v_t = np.full(x[0].shape, 0).reshape(-1, 1)
    m_t_b = 0.0
    v_t_b = 0.0
    t = 0
    epsilon = 1e-8
    
    # Training loop
    for num in range(epoch):
        for b in range(int(x.shape[0]/batch_size)):
            t+=1
            x_batch = x[b * batch_size:(b+1) * batch_size]
            y_batch = y[b * batch_size:(b+1) * batch_size].reshape(-1,1)

            # Prediction of linear regression 
            pred = np.dot(x_batch, w) + bias
            # loss
            loss = y_batch - pred
            
            # Compute gradient
            g_t = np.dot(x_batch.transpose(), loss) * (-2) +  2 * lam * np.sum(w) 
            g_t_b = loss.sum(axis=0) * (-2)
            m_t = beta_1 * m_t + (1-beta_1) * g_t 
            v_t = beta_2 * v_t + (1-beta_2) * np.multiply(g_t, g_t)
            m_cap = m_t / (1-(beta_1**t))
            v_cap = v_t / (1 - (beta_2**t))
            m_t_b = 0.9 * m_t_b + (1 - 0.9) * g_t_b
            v_t_b = 0.99 * v_t_b + (1 - 0.99) * (g_t_b * g_t_b) 
            m_cap_b = m_t_b / (1 - (0.9**t))
            v_cap_b = v_t_b / (1 - (0.99**t))
            w_0 = np.copy(w)
            
            # Update weight & bias
            w -= ((lr * m_cap) / (np.sqrt(v_cap) + epsilon)).reshape(-1, 1)
            bias -= (lr * m_cap_b) / (math.sqrt(v_cap_b) + epsilon)
            

    return w, bias

In [144]:
from argparse import Namespace

# TODO: Tune the config to boost your performance. 
train_config = Namespace(
    batch_size = 512,
    lr = 1e-1,
    lam = 0.001,
    epoch = 200,
)

# **Training your regression model**

In [157]:
# Choose your features to train. 
# Hint: 
# 1. You can select more than one feature.
# 2. You should select "good" features.

# TODO: Carefully justify which feature should be chosen.
# feats = [1, 2, 3, 4, 6, 14] # Choose CO, NO, NO2, NOx, PM2.5, PM10
# feats = [0, 5, 7, 8, 9, 10, 11, 12, 13]
feats = np.ndarray.tolist(np.arange(15))

In [146]:
# Training data preprocessing.
data = pd.read_csv("./train.csv")
data = data.values  # Data type is <class 'numpy.ndarray'>

# To transpose the data to data^T
# count = 0
train_data = np.transpose(np.array(np.float64(data)))   # Data type is <class 'numpy.ndarray'>
train_x, train_y = parse2train(train_data, feats)
# print(count)

In [147]:
# Train your regression model

w, bias = minibatch(train_x, train_y, train_config)
print(w.shape, bias.shape)

(72, 1) (1,)


# **Testing:**


In [148]:
def parse2test(data, feats):
  x = []
  for i in range(90):
    x_tmp = data[feats, 8*i:8*i+8]
    x.append(x_tmp.reshape(-1,))

  # x.shape: (n, 15, 8)
  x = np.array(x)
  return x


In [149]:
data = pd.read_csv('test.csv')
data = data.values

test_data = np.transpose(np.array(np.float64(data)))
test_x = parse2test(test_data, feats)

# **Write result as .csv**

---



In [150]:
with open('my_sol.csv', 'w', newline='') as csvf:
    # 建立 CSV 檔寫入器
    writer = csv.writer(csvf)
    writer.writerow(['Id','Predicted'])

    print(test_x.shape) 
    for i in range(int(test_x.shape[0])):
      # Prediction of linear regression 
      prediction = (np.dot(np.reshape(w,-1),test_x[i]) + bias)[0]
      writer.writerow([i, prediction] )

(90, 72)
